# Architecture A - Multi Concat Functions with Dropout and using Glover Embedding

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Architecture-A---Multi-Concat-Functions-with-Dropout-and-using-Glover-Embedding" data-toc-modified-id="Architecture-A---Multi-Concat-Functions-with-Dropout-and-using-Glover-Embedding-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Architecture A - Multi Concat Functions with Dropout and using Glover Embedding</a></span></li><li><span><a href="#Download-libraries-for-Google-Colab-and-Download-Embeddings" data-toc-modified-id="Download-libraries-for-Google-Colab-and-Download-Embeddings-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Download libraries for Google Colab and Download Embeddings</a></span></li><li><span><a href="#Import-Data" data-toc-modified-id="Import-Data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Import Data</a></span></li><li><span><a href="#Load-Embedding-and-Create-Embedding-Layer" data-toc-modified-id="Load-Embedding-and-Create-Embedding-Layer-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Load Embedding and Create Embedding Layer</a></span></li><li><span><a href="#Define-and-Train-Network" data-toc-modified-id="Define-and-Train-Network-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Define and Train Network</a></span></li><li><span><a href="#Pass-Train-and-Test-Data-Through-Network-and-Save-for-Stacking" data-toc-modified-id="Pass-Train-and-Test-Data-Through-Network-and-Save-for-Stacking-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Pass Train and Test Data Through Network and Save for Stacking</a></span></li></ul></div>

# Download libraries for Google Colab and Download Embeddings

In [1]:
! pip install pandas==0.23.4

    100% |████████████████████████████████| 8.9MB 4.0MB/s eta 0:00:01
    100% |████████████████████████████████| 13.9MB 2.9MB/s eta 0:00:01
    100% |████████████████████████████████| 512kB 9.2MB/s eta 0:00:01
twisted 18.7.0 has requirement Automat>=0.3.0, but you'll have automat 0.0.0 which is incompatible.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [0]:
import pandas as pd
pd.__version__

'0.22.0'

In [0]:
! wget http://nlp.stanford.edu/data/glove.6B.zip

--2018-12-12 20:04:00--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2018-12-12 20:04:00--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  86.0MB/s    in 9.2s    

2018-12-12 20:04:09 (89.6 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]



In [0]:
! unzip glove.6B.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


# Import Data

In [0]:
import numpy as np
import pandas as pd
import csv

In [0]:
pd.__version__

'0.23.4'

In [0]:
df_train = pd.read_csv("train_data.csv",doublequote=True,quotechar='"',sep=",").drop("is_duplicate",axis=1)

In [0]:
df_t_labels= pd.read_csv('train_labels.csv', encoding='utf-8')
df_t_labels.head()

,id,is_duplicate
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [0]:
df_train = df_train.merge(df_t_labels,on=["id"],how="inner")
df_train.head()
df_train['id'] = df_train['id'].apply(str)
df_train['is_duplicate'] = df_train['is_duplicate'].apply(int)

In [0]:
df_train['question1'].fillna('', inplace=True)
df_train['question2'].fillna('', inplace=True)

In [0]:
pd.__version__

'0.23.4'

In [0]:
df_test = pd.read_csv('test_data.csv',doublequote=True)
df_test['test_id'] = df_test['test_id'].apply(str)

In [0]:
df_all = pd.concat((df_train, df_test))
df_all['question1'].fillna('', inplace=True)
df_all['question2'].fillna('', inplace=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [0]:
from sklearn.feature_extraction.text import CountVectorizer
import itertools

In [0]:
counts_vectorizer = CountVectorizer(max_features=10000-1).fit(
    itertools.chain(df_all['question1'], df_all['question2']))

other_index = len(counts_vectorizer.vocabulary_)

In [0]:
import re
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import keras 
keras.__version__

Using TensorFlow backend.


'2.2.4'

In [0]:
words_tokenizer = re.compile(counts_vectorizer.token_pattern)

In [0]:
def create_padded_seqs(texts, max_len=30):
    seqs = texts.apply(lambda s: 
        [counts_vectorizer.vocabulary_[w] if w in counts_vectorizer.vocabulary_ else other_index
         for w in words_tokenizer.findall(s.lower())])
    return pad_sequences(seqs, maxlen=max_len)

In [0]:
nlp_train_df = pd.read_csv("train_features_31.csv")

In [0]:
print(nlp_train_df.drop("id",axis=1).shape)
print(nlp_train_df.shape)

(323164, 31)
(323164, 32)


In [0]:
X1_train, X1_val, X2_train, X2_val, y_train, y_val, train_id, train_val_id, nlp_train, nlp_val= \
    train_test_split(create_padded_seqs(df_train['question1']),
                     create_padded_seqs(df_train['question2']),
                     df_train['is_duplicate'].values,
                     df_train.id,
                     nlp_train_df.drop("id",axis=1).as_matrix(),
                     stratify=df_train['is_duplicate'].values,
                     test_size=0.3, random_state=2018)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """


In [0]:
train_id_pd = pd.DataFrame(train_id.values)
train_val_id_pd = pd.DataFrame(train_val_id.values)

In [0]:
train_id_pd.to_csv("train_id_multiple_merge_dropout.csv",index=False)
train_val_id_pd.to_csv("train_val_id_multiple_merge_dropout.csv", index=False)

# Load Embedding and Create Embedding Layer

In [0]:
import os
import numpy as np

GLOVE_DIR = ""

embeddings_index = {}
line_num = 0

f = open(os.path.join(GLOVE_DIR, 'glove.6B.300d.txt'))
ff = list(f)
f.close()

for line in ff:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
    line_num += 1
    if line_num == 400000:
        break;

In [0]:
EMBEDDING_DIM = 300

word_index = counts_vectorizer.vocabulary_

embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))

for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [0]:
from keras.layers import Embedding

MAX_SEQUENCE_LENGTH = X1_train.shape[1]

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

# Define and Train Network

In [0]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

In [0]:
from keras.layers import *
from keras.models import Model

In [0]:
words_embedding_layer = embedding_layer
seq_embedding_layer = LSTM(256, activation='tanh')

input1_tensor = Input(X1_train.shape[1:])
input2_tensor = Input(X2_train.shape[1:])

words_embedding_layer1 = words_embedding_layer(input1_tensor)
words_embedding_layer2 = words_embedding_layer(input2_tensor)

seq_embedding_layer1 = seq_embedding_layer(words_embedding_layer1) 
seq_embedding_layer2 = seq_embedding_layer(words_embedding_layer2)

multiply_merge = multiply([seq_embedding_layer1, seq_embedding_layer2])

sub_merge = subtract([seq_embedding_layer1, seq_embedding_layer2])

add_merge = add([seq_embedding_layer1, seq_embedding_layer2])

max_merge = maximum([seq_embedding_layer1, seq_embedding_layer2])

merge_layer = concatenate([multiply_merge, sub_merge, add_merge,max_merge])

dropout1 = Dropout(0.15)(merge_layer)

dense1_layer = Dense(100, activation='sigmoid')(dropout1)

dropout2 = Dropout(0.15)(dense1_layer)

dense2_layer = Dense(100, activation='sigmoid')(dropout2)


ouput_layer = Dense(1, activation='sigmoid')(dense2_layer)

model = Model([input1_tensor, input2_tensor], ouput_layer)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=["accuracy"])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 30)           0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, 30)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 30, 300)      3000000     input_5[0][0]                    
                                                                 input_6[0][0]                    
__________________________________________________________________________________________________
lstm_3 (LSTM)                   (None, 256)          570368      embedding_1[4][0]                
          

In [0]:
# with 300d
#Epoch 00003: val_loss improved from 0.42090 to 0.39988, saving model to weights.best.Siamese.gloverLSTM256
#Epoch 4/5
#226214/226214 [==============================] - 329s 1ms/step - loss: 0.3274 - acc: 0.8537 - val_loss: 0.3956 - val_acc: 0.8191

In [0]:
! mkdir weights.LSTM256.multiple.merge

mkdir: cannot create directory ‘weights.LSTM256.multiple.merge’: File exists


In [0]:
from keras.callbacks import ModelCheckpoint  


directory = "weights.LSTM256.multiple.merge/dropout.{epoch:02d}-{loss:.2f}-{val_loss:.2f}.hdf5"

checkpointer = ModelCheckpoint(filepath=directory, 
                               verbose=1, save_best_only=False,monitor="val_loss")


In [0]:
model.fit([X1_train, X2_train], 
          y_train, 
            batch_size=128, 
            epochs=5,
            callbacks=[checkpointer],
            validation_data=([X1_val, X2_val], y_val))

Train on 226214 samples, validate on 96950 samples
Epoch 1/5
226214/226214 [==============================] - 314s 1ms/step - loss: 0.4712 - acc: 0.7670 - val_loss: 0.4171 - val_acc: 0.8005

Epoch 00001: saving model to weights.LSTM256.multiple.merge/dropout.01-0.47-0.42.hdf5
Epoch 2/5
226214/226214 [==============================] - 511s 2ms/step - loss: 0.3835 - acc: 0.8205 - val_loss: 0.3876 - val_acc: 0.8181

Epoch 00002: saving model to weights.LSTM256.multiple.merge/dropout.02-0.38-0.39.hdf5
Epoch 3/5
226214/226214 [==============================] - 871s 4ms/step - loss: 0.3354 - acc: 0.8477 - val_loss: 0.3732 - val_acc: 0.8273

Epoch 00003: saving model to weights.LSTM256.multiple.merge/dropout.03-0.34-0.37.hdf5
Epoch 4/5
226214/226214 [==============================] - 883s 4ms/step - loss: 0.2912 - acc: 0.8707 - val_loss: 0.3755 - val_acc: 0.8323

Epoch 00004: saving model to weights.LSTM256.multiple.merge/dropout.04-0.29-0.38.hdf5
Epoch 5/5
226214/226214 [====================

In [0]:
#gloverd100
#dropout
#multi-merge

#Train on 226214 samples, validate on 96950 samples
#Epoch 1/5
#226214/226214 [==============================] - 314s 1ms/step - loss: 0.4712 - acc: 0.7670 - val_loss: 0.4171 - val_acc: 0.8005
#
#Epoch 00001: saving model to weights.LSTM256.multiple.merge/dropout.01-0.47-0.42.hdf5
#Epoch 2/5
#226214/226214 [==============================] - 511s 2ms/step - loss: 0.3835 - acc: 0.8205 - val_loss: 0.3876 - val_acc: 0.8181
#
#Epoch 00002: saving model to weights.LSTM256.multiple.merge/dropout.02-0.38-0.39.hdf5
#Epoch 3/5
#226214/226214 [==============================] - 871s 4ms/step - loss: 0.3354 - acc: 0.8477 - val_loss: 0.3732 - val_acc: 0.8273
#
#Epoch 00003: saving model to weights.LSTM256.multiple.merge/dropout.03-0.34-0.37.hdf5
#Epoch 4/5
#226214/226214 [==============================] - 883s 4ms/step - loss: 0.2912 - acc: 0.8707 - val_loss: 0.3755 - val_acc: 0.8323
#
#Epoch 00004: saving model to weights.LSTM256.multiple.merge/dropout.04-0.29-0.38.hdf5
#Epoch 5/5
#226214/226214 [==============================] - 887s 4ms/step - loss: 0.2490 - acc: 0.8916 - val_loss: 0.3866 - val_acc: 0.8340
#
#Epoch 00005: saving model to weights.LSTM256.multiple.merge/dropout.05-0.25-0.39.hdf5
#<keras.callbacks.History at 0x7f35ece05b00>

In [0]:
# Load Best Epoch
model.load_weights("weights.LSTM256.multiple.merge/dropout.02-0.38-0.39.hdf5")

# Pass Train and Test Data Through Network and Save for Stacking

In [0]:
features_model_16 = Model([input1_tensor, input2_tensor], dense1_layer)
features_model_16.compile(loss='mse', optimizer='adam')

In [0]:
train_preds = [create_padded_seqs(df_train['question1']),create_padded_seqs(df_train['question2'])]
test_preds = [create_padded_seqs(df_test['question1']),create_padded_seqs(df_test['question2'])]

In [0]:
F_train_16 = features_model_16.predict(train_preds, batch_size=128)
F_test_16 = features_model_16.predict(test_preds, batch_size=128)

In [0]:
F_train_1 = model.predict(train_preds, batch_size=128)
F_test_1 = model.predict(test_preds, batch_size=128)

In [0]:
test_1 = pd.concat([df_test.test_id,pd.DataFrame(F_test_1)],axis=1)
test_16 = pd.concat([df_test.test_id,pd.DataFrame(F_test_16)],axis=1)

In [0]:
print(test_1.shape)
print(df_test.shape)
test_16.head(1)

(81126, 2)
(81126, 3)


,test_id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,...,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
0,15,0.800644,0.011924,0.887471,0.821151,0.005397,0.052657,0.909325,0.003853,0.027626,0.909195,0.015163,0.797268,0.946934,0.010175,0.847477,0.793004,0.014024,0.017926,0.011315,0.013858,0.012912,0.05535,0.669011,0.016277,0.057484,0.009332,0.042779,0.010491,0.016269,0.018689,0.552569,0.830185,0.008974,0.002655,0.923107,0.862485,0.751753,0.015057,0.903913,...,0.056969,0.013762,0.008884,0.558022,0.702742,0.914065,0.778544,0.027683,0.017505,0.072732,0.767495,0.851849,0.028745,0.955579,0.009185,0.012054,0.842147,0.763231,0.020196,0.040096,0.017247,0.683329,0.036683,0.845951,0.009065,0.008742,0.061022,0.012774,0.040569,0.018408,0.726537,0.029499,0.010284,0.711195,0.576707,0.929686,0.539161,0.804008,0.848154,0.033959


In [0]:
train_1 = pd.concat([df_train.id,pd.DataFrame(F_train_1)],axis=1)
train_16 = pd.concat([df_train.id,pd.DataFrame(F_train_16)],axis=1)

In [0]:
print(train_1.shape)
print(df_train.shape)
train_16.head(1)

(323164, 2)
(323164, 4)


,id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,...,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
0,0,0.0529,0.025055,0.318428,0.50657,0.020913,0.023034,0.675262,0.016047,0.018225,0.629039,0.013483,0.603198,0.006767,0.016828,0.302478,0.741219,0.049589,0.087637,0.137847,0.033844,0.030675,0.180749,0.153638,0.02582,0.140611,0.057422,0.082687,0.036608,0.033327,0.048423,0.080166,0.185999,0.012485,0.015729,0.284244,0.637421,0.463412,0.023377,0.364163,...,0.127946,0.038022,0.024877,0.489653,0.1676,0.734839,0.329826,0.125758,0.042801,0.109782,0.357218,0.330495,0.045412,0.155905,0.021132,0.024861,0.357092,0.149614,0.030009,0.04164,0.02448,0.13045,0.020895,0.410603,0.020111,0.01846,0.197218,0.046063,0.054779,0.018068,0.337488,0.059948,0.07311,0.445842,0.241049,0.592366,0.009818,0.539237,0.533372,0.063803


In [0]:
train_16.to_csv("train_16_multi_merge_dropout.csv",index=False)
train_1.to_csv("train_1_merge_dropout.csv",index=False)

In [0]:
pd.read_csv("train_16_multi_merge_dropout.csv").head()

,id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,...,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
0,0,0.052900,0.025055,0.318428,0.506570,0.020913,0.023034,0.675262,0.016047,0.018225,0.629039,0.013483,0.603198,0.006767,0.016828,0.302478,0.741219,0.049589,0.087637,0.137847,0.033844,0.030675,0.180749,0.153638,0.025820,0.140611,0.057422,0.082687,0.036608,0.033327,0.048423,0.080166,0.185999,0.012485,0.015729,0.284244,0.637421,0.463412,0.023377,0.364163,...,0.127946,0.038022,0.024877,0.489653,0.167600,0.734840,0.329826,0.125758,0.042801,0.109782,0.357218,0.330495,0.045412,0.155905,0.021132,0.024861,0.357092,0.149614,0.030009,0.041640,0.024480,0.130450,0.020895,0.410603,0.020111,0.018460,0.197218,0.046063,0.054779,0.018068,0.337488,0.059948,0.073110,0.445842,0.241049,0.592366,0.009818,0.539237,0.533372,0.063803
1,1,0.013695,0.145779,0.079324,0.071615,0.215110,0.241945,0.076339,0.193873,0.115941,0.122648,0.294601,0.106935,0.002617,0.327100,0.090014,0.017598,0.278505,0.621992,0.211655,0.583829,0.383984,0.283039,0.029181,0.464668,0.754974,0.344025,0.454034,0.218219,0.419860,0.282790,0.018985,0.028972,0.214981,0.293527,0.018983,0.053465,0.030082,0.469298,0.071777,...,0.575017,0.293818,0.198420,0.012704,0.030329,0.018374,0.012731,0.812980,0.449378,0.576248,0.050341,0.030157,0.225895,0.216638,0.387833,0.545361,0.091207,0.025223,0.675127,0.164489,0.172073,0.011124,0.625859,0.051404,0.377757,0.262307,0.452393,0.235349,0.328871,0.248712,0.021916,0.590705,0.753402,0.025976,0.022266,0.193322,0.001406,0.069366,0.059980,0.576983
2,2,0.072572,0.104996,0.354583,0.491617,0.076595,0.110560,0.140739,0.132093,0.139132,0.272483,0.260845,0.355731,0.000469,0.419366,0.217963,0.244305,0.191362,0.202435,0.248744,0.217868,0.370406,0.211159,0.029961,0.271763,0.636783,0.130924,0.286630,0.190934,0.068344,0.767804,0.123639,0.109012,0.116066,0.224112,0.047043,0.171371,0.274246,0.117537,0.066214,...,0.365381,0.175198,0.233734,0.230603,0.154447,0.125233,0.182839,0.446325,0.805794,0.380621,0.127164,0.208606,0.426908,0.020711,0.238079,0.071783,0.081577,0.067003,0.199035,0.223704,0.207602,0.063678,0.223355,0.209076,0.189759,0.204944,0.341722,0.204727,0.348571,0.318086,0.177841,0.183011,0.207380,0.257704,0.115585,0.271708,0.007019,0.106703,0.306342,0.382994
3,3,0.047407,0.856744,0.066668,0.046414,0.654683,0.442411,0.044354,0.841770,0.719654,0.133001,0.639230,0.065401,0.003770,0.656466,0.061181,0.018879,0.713081,0.710977,0.652039,0.802149,0.713393,0.738163,0.024050,0.825086,0.754566,0.846251,0.842834,0.746928,0.865909,0.682754,0.021553,0.042277,0.789656,0.848372,0.038664,0.156200,0.029353,0.817556,0.014326,...,0.654965,0.583945,0.717332,0.035134,0.060924,0.036549,0.049155,0.811794,0.847691,0.635823,0.027131,0.040872,0.774282,0.020331,0.877778,0.929009,0.106039,0.019663,0.789611,0.686795,0.672481,0.025384,0.702995,0.051921,0.598549,0.758062,0.665601,0.862356,0.805011,0.689778,0.035593,0.665647,0.749958,0.017688,0.026764,0.086354,0.003828,0.036413,0.112572,0.736643
4,4,0.012602,0.711812,0.071569,0.044851,0.708731,0.362941,0.040776,0.480433,0.584040,0.060250,0.736261,0.048488,0.032048,0.904479,0.064298,0.022738,0.540619,0.562304,0.881918,0.842282,0.710248,0.709233,0.019294,0.712840,0.934187,0.754544,0.737463,0.868673,0.311979,0.928508,0.030664,0.060964,0.767515,0.602867,0.031160,0.023415,0.066440,0.631997,0.019464,...,0.929188,0.746774,0.675822,0.007927,0.038571,0.025504,0.008351,0.927918,0.827824,0.918143,0.045404,0.038814,0.835670,0.049485,0.655468,0.537027,0.034726,0.027015,0.569623,0.785604,0.898315,0.018262,0.839751,0.032901,0.721045,0.605482,0.948713,0.570968,0.847235,0.794699,0.029819,0.919816,0.827702,0.027208,0.023328,0.057312,0.008514,0.025699,0.025907,0.630518


In [0]:
test_16.to_csv("test_16_multi_merge_dropout.csv",index=False)
test_1.to_csv("test_1_multi_merge_dropout.csv",index=False)